In [4]:
%%writefile serial.cpp
#include <iostream>
using namespace std;

int main(){
  int i; nb_rec = 10000000;
  double x, n, sum = 0.0, pi 0.0;
  h = 1.0 / nb_rec;
  for (i = 0; i<nb_rec; i++){
    x = (i+0.5)*h;
    sum += 4.0 / (1.0 + x*x);
  }
  pi = h * sum;
  cout << " O Valor aproximado de PI é: " << pi << endl;
  return 0;
}

Writing serial.cpp


In [ ]:
!g++ -Wall -O3 serial.cpp -o serial

In [6]:
%%writefile serial.slurm
#!/bin/bash
#SBATCH -N 1
#SBATCH --time=00:10:00
#SBATCH --partition=normal

./serial

Writing serial.slurm


In [7]:
%%writefile point_to_point.cpp
#include <iostream>
#include <mpi.h>
using namespace std;

int main(){
  int i, nb_rec = 10000000;
  double x, h, pi =  0.0;
  int myRank, size, tag = 10;
  int retangulos;
  int meu_inicio=0, meu_fim=0;
  double minha_soma = 0.0;
  MPI_Init(NULL,NULL);
  MPI_Comm_rank(MPI_COMM_WORLD,&myRank);
  MPI_Comm_size(MPI_COMM_WORLD,&size);
  MPI_Status status;
  if (myRank == 0) {
      retangulos = nb_rec / size;
      for (i = 1; i < size; i++){
        MPI_Send(&retangulos, 1, MPI_INT, i, tag, MPI_COMM_WORLD);
      }

  }
  else{
    MPI_Recv(&retangulos,1, MPI_INT, 0 ,tag, MPI_COMM_WORLD,&status);
  }

  h = 1.0 / (retangulos * size);
  meu_inicio = myRank * retangulos;
  meu_fim = meu_inicio+retangulos;
  for (i = meu_inicio; i < meu_fim; i++){
    x = (i+0.5)*h;
    minha_soma += 4.0 / (1.0+x*x);
  }
  if (myRank != 0){
    MPI_Send(&minha_soma,1,MPI_DOUBLE,0,tag,MPI_COMM_WORLD);
  }
  else{
    for (i = 1; i<size;i++){
      MPI_Recv(&pi,1,MPI_DOUBLE,MPI_ANY_SOURCE,tag,MPI_COMM_WORLD,&status);
      minha_soma = minha_soma + pi;
    }
    pi = h* minha_soma;
    cout << " O valor aproximado de  PI é: " << pi << endl;
  }
  MPI_Finalize();
  return 0;
}

Overwriting point_to_point.cpp


In [8]:
!mpic++ -o point_to_point point_to_point.cpp

In [9]:
%%writefile point_to_point.slurm
#!/bin/bash
#SBATCH -n 5

mpirun --allow-run-as-root -np 5 ./point_to_point

Writing point_to_point.slurm


In [10]:
%%writefile broadcast.cpp
#include <iostream>
#include <mpi.h>
using namespace std;

int main(){
  int i, nb_rec = 10000000;
  double x, h, pi =  0.0;
  int myRank, size, tag = 10;
  int retangulos;
  int meu_inicio, meu_fim;
  double minha_soma = 0.0;
  MPI_Init(NULL,NULL);
  MPI_Comm_rank(MPI_COMM_WORLD,&myRank);
  MPI_Comm_size(MPI_COMM_WORLD,&size);
  MPI_Status status;
  if (myRank == 0) {
      retangulos = nb_rec / size;
  }
  MPI_Bcast(&retangulos,1,MPI_INT,0,MPI_COMM_WORLD);

  h = 1.0 / (retangulos * size);
  pi = h*minha_soma;
  meu_inicio = myRank * retangulos;
  meu_fim = meu_inicio+retangulos;
  for (i = meu_inicio; i < meu_fim; i++){
    x = (i+0.5)*h;
    minha_soma += 4.0 / (1.0+x*x);
  }
 minha_soma = h*minha_soma;
  MPI_Reduce(&minha_soma, &pi, 1, MPI_DOUBLE, MPI_SUM, 0, MPI_COMM_WORLD);
    if (myRank == 0)
    cout << " O valor aproximado de  PI é: " << pi << endl;
  MPI_Finalize();
  return 0;
}

Overwriting broadcast.cpp


In [11]:
!mpic++ -o broadcast broadcast.cpp

In [12]:
%%writefile broadcast.slurm
#!/bin/bash
#SBATCH -n 5

mpirun --allow-run-as-root -np 5 ./broadcast

Writing broadcast.slurm
